In [ ]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import pandas as pd
import pickle

# Leitura dos dados de treino
df_treino_svm = pd.read_csv('./data/Train.csv')

# Inicialização do CountVectorizer
cv_svm = CountVectorizer(lowercase=True, stop_words='english', ngram_range=(1, 1))

# Transformação dos dados de treino
X_treino_cv_svm = cv_svm.fit_transform(df_treino_svm['text'])
y_treino_svm = df_treino_svm['label']

# Inicialização do modelo SVM
svm_model = SVC(random_state=42)

# Definindo o espaço de hiperparâmetros para o Grid Search
param_grid_svm = {
    'C': [0.1, 1, 10],  # Regularização
    'kernel': ['rbf', 'linear', 'sigmoid'],  # Tipo de kernel
    'gamma': ['scale', 'auto']  # Coeficiente de kernel para 'rbf', 'poly' e 'sigmoid'
}

# Inicialização do Grid Search com validação cruzada
grid_search_svm = GridSearchCV(
    svm_model,
    param_grid_svm,
    n_jobs=4,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=5),
)
grid_search_svm.fit(X_treino_cv_svm, y_treino_svm)

# Imprimindo o melhor conjunto de hiperparâmetros
print("Melhores hiperparâmetros para SVM: ", grid_search_svm.best_params_)

# Treinamento do modelo com o melhor conjunto de hiperparâmetros
best_svm_model = grid_search_svm.best_estimator_

# Função para carregar e pré-processar os dados IMDB
def load_and_preprocess_IMDB(filename, nrows=None):
    """ load the IMDB data and preprocess it:
            - remove html tags
            - remove ponctuation
            - convert to lower case
            - remove stop words
            - remove numbers
            - remove extra spaces
            - replave words with their root form (stem)
            - replace words with their lemma
        :param filename: 'train' or 'test'
        :param nrows: number of rows to read
        :return: df
    """

    # read the data
    df = pd.read_csv(filename, nrows=nrows)

    # keep a copy of the original text
    df['original_text'] = df['text']

    # remove the html tags
    df['text'] = df['text'].str.replace('<br />', ' ')

    # remove the punctuation and '_' characters
    df['text'] = df['text'].str.replace('[^\w\s]', ' ', regex=True)
    df['text'] = df['text'].str.replace('_', ' ', regex=False)

    # convert to lower case
    df['text'] = df['text'].str.lower()

    # remove the stop words
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

    # remove the numbers - test on https://regexr.com
    df['text'] = df['text'].str.replace('\d+', '', regex=True)

    # remove the extra spaces - test on https://regexr.com
    df['text'] = df['text'].str.replace(' +', ' ', regex=True)

    # replace the words with their root form
    from nltk.stem import SnowballStemmer
    stemmer = SnowballStemmer('english')
    df['text'] = df['text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

    # replace the words with their lemma
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    df['text'] = df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

    return df

# Leitura e pré-processamento dos dados de teste
df_teste_svm = load_and_preprocess_IMDB('./data/Test.csv')
count_vectors_teste_svm = cv_svm.transform(df_teste_svm['text'])
bow_teste_svm = pd.DataFrame(count_vectors_teste_svm.toarray(), columns=cv_svm.get_feature_names_out())

# Avaliação do modelo nos dados de teste
acuracia_svm = best_svm_model.score(bow_teste_svm, df_teste_svm['label'])
print(f"Acurácia do SVM nos dados de teste com hiperparâmetros otimizados: {acuracia_svm}")

# Salvando o modelo e o vetorizador no disco
filename_svm_model = 'svm_model.sav'
pickle.dump(best_svm_model, open(filename_svm_model, 'wb'))

filename_vectorizer_svm = 'svm_count_vectorizer.sav'
pickle.dump(cv_svm, open(filename_vectorizer_svm, 'wb'))
